In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [3]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [6]:
load_dotenv()

True

In [7]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7
)


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [8]:
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

In [10]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

chatbot.invoke(initial_state)

{'messages': [HumanMessage(content='What is the capital of india', additional_kwargs={}, response_metadata={}, id='42e2498a-67ab-4cec-b95e-f4dfc8f99168'),
  AIMessage(content='The capital of India is New Delhi.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 41, 'total_tokens': 50, 'completion_time': 0.006579546, 'completion_tokens_details': None, 'prompt_time': 0.002971271, 'prompt_tokens_details': None, 'queue_time': 0.070070139, 'total_time': 0.009550817}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4d3c-37b8-7f22-9458-790d024d70c6-0', usage_metadata={'input_tokens': 41, 'output_tokens': 9, 'total_tokens': 50})]}

In [13]:
while True:
  user_message = input("You: ")

  if user_message.lower() in ['exit', 'quit']:
      print("Exiting chat.")
      break
  response = chatbot.invoke({
      'messages': [HumanMessage(content=user_message)]})
  print(response)

{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='757ad2f8-fc80-421c-8106-5a0d1afe9945'), AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.011037222, 'completion_tokens_details': None, 'prompt_time': 0.001638697, 'prompt_tokens_details': None, 'queue_time': 0.050510423, 'total_time': 0.012675919}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b4d42-2fe8-7f31-9cc7-c7c62fbb94a0-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})]}
{'messages': [HumanMessage(content='i want to study quantum fluid entaglement', additional_kwargs={}, response_metadata={}, id='0aa51cfe-0a3b-41f9-a5b2-f21a22bed8bd'), AIMessage(content='Quantum flui